### importing the dataset from kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/Kaggle/kaggle.json' '/content'

In [ ]:
import zipfile
import os

os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

 99% 171M/172M [00:09<00:00, 20.5MB/s]
100% 172M/172M [00:10<00:00, 17.9MB/s]


In [ ]:
zip_ref = zipfile.ZipFile('bone-fracture-detection-using-xrays.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:

len(os.listdir('/tmp/archive (6)/train'))

2

### Preprocessing images


In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

In [ ]:
Train_generator = generator.flow_from_directory(
    "/tmp/archive (6)/train",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "training"
)

Validation_generator = generator.flow_from_directory(
    "/tmp/archive (6)/train",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "validation"
)

Found 7091 images belonging to 2 classes.
Found 1772 images belonging to 2 classes.


In [ ]:
Train_generator.next()[0].shape

(16, 224, 224, 3)

In [ ]:
Validation_generator.next()[0].shape

(16, 224, 224, 3)

### Xception

In [ ]:
from tensorflow import keras

In [ ]:
base_model = keras.applications.Xception(
    weights=None,
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
base_model.trainable = True

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=True)
x = keras.layers.Dropout(0.80)(x)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.GaussianNoise(0.80)(x)
x = keras.layers.Dense(512,activation='relu',kernel_regularizer=keras.regularizers.l2(0.50))(x)
x = keras.layers.GaussianNoise(0.80)(x)
x = keras.layers.Dropout(0.80)(x)
x = keras.layers.BatchNormalization()(x)
outputs = keras.layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)

In [ ]:
from keras import callbacks

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(
    patience=7,
)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.compile(optimizer= "adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.fit(Train_generator, epochs=20,
          callbacks=[earlyStopping,model_checkpoint_callback ],
          validation_data=Validation_generator)

Epoch 1/20
444/444 [==============================] - 201s 346ms/step - loss: 13.4442 - accuracy: 0.9315 - val_loss: 2.3141 - val_accuracy: 0.5739
Epoch 2/20
444/444 [==============================] - 154s 347ms/step - loss: 0.8978 - accuracy: 0.9631 - val_loss: 2.3137 - val_accuracy: 0.5954
Epoch 3/20
104/444 [======>.......................] - ETA: 1:36 - loss: 0.9791 - accuracy: 0.9694

In [ ]:
model.summary()

In [ ]:
model.evaluate(Validation_generator)

In [ ]:
Test_Generator = generator.flow_from_directory(
    "/tmp/archive (6)/val",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "training"
)

In [ ]:
model.evaluate(Test_Generator)

### ResNet

In [ ]:
from tensorflow import keras

In [ ]:
base_model = keras.applications.ResNet101(
    weights=None,
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
base_model.trainable = True

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=True)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(512,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
outputs = keras.layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)

In [ ]:
from keras import callbacks

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(
    patience=7,
)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.compile(optimizer= "adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.fit(Train_generator, epochs=5,
          callbacks=[earlyStopping,model_checkpoint_callback ],
          validation_data=Validation_generator)

In [ ]:
model.summary()

In [ ]:
model.evaluate(Train_generator)

In [ ]:
Test_Generator = generator.flow_from_directory(
    "/tmp/archive (6)/val",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "training"
)

In [ ]:
model.evaluate(Test_Generator)

### VGG 19

In [ ]:
from tensorflow import keras

In [ ]:
base_model = keras.applications.VGG19(
    weights=None,
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
base_model.trainable = True

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=True)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(512,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
outputs = keras.layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)

In [ ]:
from keras import callbacks

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(
    patience=7,
)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.compile(optimizer= "adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.fit(Train_generator, epochs=5,
          callbacks=[earlyStopping,model_checkpoint_callback ],
          validation_data=Validation_generator)

In [ ]:
model.summary()

In [ ]:
model.evaluate(Train_generator)

In [ ]:
Test_Generator = generator.flow_from_directory(
    "/tmp/archive (6)/val",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "training"
)

In [ ]:
model.evaluate(Test_Generator)